Setup
1. Use GPU

Make sure you are using a GPU runtime. Click on runtime -> change runtime type. Under hardware accelerator select GPU and click SAVE.

2. Update fastai

Fastai comes pre-installed in google colab but its an older version. So, we will first update fastai

In [1]:
!pip install fastai --upgrade --quiet


     |████████████████████████████████| 194kB 8.3MB/s 
     |████████████████████████████████| 61kB 7.3MB/s 


In [2]:
from fastai.vision.all import *

**Building state-of-the-art Image classifier**


**Collecting data**


Lets start by creating a data folder where we will keep all our data.

In [3]:
path=Path('Data')
path.mkdir(exist_ok=True)

In [4]:

!wget -q https://raw.githubusercontent.com/Manjubn777/Flower-classifier/main/datasets/Hibiscus.txt


In [5]:
!wget -q https://raw.githubusercontent.com/Manjubn777/Flower-classifier/main/datasets/Lilium.txt
!wget -q https://raw.githubusercontent.com/Manjubn777/Flower-classifier/main/datasets/Tulip.txt

In [6]:
download_images('Data/Hibiscus', Path('Hibiscus.txt'))

In [7]:
download_images('Data/Lilium', Path('Lilium.txt'))
download_images('Data/Tulip', Path('Tulip.txt'))

In [8]:
files= get_image_files(path)
len(files)

231

99*3 = 297 images should be there but actually we are getting 231 this is due to there some extensions which can be rendered from browser but not from python eg : webpi is only rendered and tehier extensions not counted so they easily removed

#verify images


In [9]:
files
#it is list, every element pointing to an images

(#231) [Path('Data/Hibiscus/00000030.jpg'),Path('Data/Hibiscus/00000180.jpg'),Path('Data/Hibiscus/00000062.jpg'),Path('Data/Hibiscus/00000004.jpg'),Path('Data/Hibiscus/00000112.jpg'),Path('Data/Hibiscus/00000010.Jpg'),Path('Data/Hibiscus/00000056.png'),Path('Data/Hibiscus/00000082.jpg'),Path('Data/Hibiscus/00000196.jpg'),Path('Data/Hibiscus/00000158.jpg')...]

In [10]:
failed= verify_images(files)
failed

(#1) [Path('Data/Lilium/00000118.jpg')]

verify_images iterates along with all 320+images. try to read image and if it unable to read an images then its corrupted one and it wll add the name to list

In [11]:
#delete the failed images
failed.map(Path.unlink)

(#1) [None]

In [12]:
files= get_image_files(path)
len(files)

230

DataLoaders
In machine learning, almost all algorithms take the complete dataset while training. But in-case of deep learning, you don't pass the complete data at once.

You divide the data into smaller batches and pass the batches as input to deep learnining model. DataLoaders allows us to train models on huge dataset. It parallely load the data, in batches, while the model is training.

But we don't have to worry about it. The Datablocks API will take care of everything for us.

To turn our downloaded data into a DataLoaders object we need to tell fastai at least four things:

What kinds of data we are working with
How to get the list of items
How to label these items
How to create the validation set
This is how we can create a DataLoaders for the dataset that we just downloaded:

In [13]:
flowers = DataBlock(
    blocks=(ImageBlock, CategoryBlock), # x,y
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms())

Let's look at each of these arguments in turn. First we provide a tuple where we specify what types we want for the independent and dependent variables:

blocks=(ImageBlock, CategoryBlock)
The independent variable is the thing we are using to make predictions from, and the dependent variable is our target. In this case, our independent variables are images, and our dependent variables are the categories (type of animal) for each image.

For this DataLoaders our underlying items will be file paths. We have to tell fastai how to get a list of those files. The get_image_files function takes a path, and returns a list of all of the images in that path (recursively, by default):

get_items=get_image_files
Next, we will randomly split our data into training and validation sets. However, we would like to have the same training/validation split each time we run this notebook, so we fix the random seed (computers don't really know how to create random numbers at all, but simply create lists of numbers that look random; if you provide the same starting point for that list each time—called the seed—then you will get the exact same list each time):

splitter=RandomSplitter(valid_pct=0.2, seed=42)
The independent variable is often referred to as x and the dependent variable is often referred to as y. Here, we are telling fastai what function to call to create the labels in our dataset:

get_y=parent_label
parent_label function simply gets the name of the folder a file is in. Because we put each of our images into folders based on its class, this is going to give us the labels that we need.

Our images are all different sizes, and this is a problem for deep learning: we don't feed the model one image at a time but several of them (what we call a mini-batch). To group them in a big array (usually called a tensor) that is going to go through our model, they all need to be of the same size. So, we need to add a transform which will resize these images to the same size. Item transforms are pieces of code that run on each individual item, whether it be an image, category, or so forth. Here, we'll use RandomResizedCrop with an image size of 224 px, which is fairly standard for image classification, and default aug_transforms:

item_tfms=RandomResizedCrop(224, min_scale=0.5),
batch_tfms=aug_transforms()
This command has given us a DataBlock object. This is like a template for creating a DataLoaders. We still need to tell fastai the actual source of our data—in this case, the path where the images can be found:



In [14]:
Image.open(files[1])

In [15]:
files[0]

Path('Data/Hibiscus/00000030.jpg')

In [16]:
str(files[0]).split('/')

['Data', 'Hibiscus', '00000030.jpg']

In [17]:
str(files[0]).split('/')[-2]

'Hibiscus'

In [18]:
dls = flowers.dataloaders(path)

In [19]:
dls.train.show_batch(max_n=6, nrows=1)

In [20]:
learn = cnn_learner(dls, resnet50, metrics=error_rate)
learn.fine_tune(5)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


epoch,train_loss,valid_loss,error_rate,time
0,1.865880,0.744260,0.282609,00:09


epoch,train_loss,valid_loss,error_rate,time
0,0.394867,0.329936,0.130435,00:08
1,0.322321,0.292696,0.108696,00:08
2,0.223762,0.267622,0.086957,00:08
3,0.176410,0.194512,0.043478,00:08
4,0.145664,0.175045,0.043478,00:08


In [21]:
#learn = cnn_learner(dls, resnet18, metrics=error_rate)
#learn.fine_tune(3)

In [22]:
files[10]

Path('Data/Hibiscus/00000070.jpg')

In [23]:
files[71]

Path('Data/Hibiscus/00000048.jpg')

In [24]:
learn.predict(files[71])

('Hibiscus', tensor(0), tensor([9.9962e-01, 6.6939e-05, 3.1327e-04]))

In [25]:
learn.predict(files[120])

('Lilium', tensor(1), tensor([0.0279, 0.9637, 0.0085]))

In [26]:
dls.vocab

['Hibiscus', 'Lilium', 'Tulip']

In [27]:
learn.show_results()

In [28]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [29]:

interp.plot_top_losses(5, nrows=1)

Deploying the Model

First, we will export the model that we trained.

In [30]:
learn.export('export.pkl')

In [31]:

!pip install streamlit --quiet
!pip install colab-everything --quiet

     |████████████████████████████████| 7.5MB 8.1MB/s 
     |████████████████████████████████| 81kB 13.5MB/s 
     |████████████████████████████████| 163kB 61.8MB/s 
     |████████████████████████████████| 4.5MB 59.7MB/s 
     |████████████████████████████████| 112kB 64.0MB/s 
     |████████████████████████████████| 71kB 9.9MB/s 
     |████████████████████████████████| 122kB 64.9MB/s 
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.4.3 which is incompatible.
     |████████████████████████████████| 133kB 7.0MB/s 
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.4.3 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.


In [32]:
!wget -q https://raw.githubusercontent.com/Manjubn777/Flower-classifier/main/app.py

In [ ]:
from colab_everything import ColabStreamlit
ColabStreamlit('app.py')

Web App can be accessed on: http://7fc821c45a83.ngrok.io

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:9999
  External URL: http://35.185.220.44:9999

